In [1]:
import pandas as pd
import plotly.express as px
import json
import plotly.graph_objects as go

In [71]:
new = pd.read_csv('results.csv', index_col=0)
paper_results = pd.read_excel('data/mdevs_solutions.xlsx', sheet_name=["results_regular_BCH", "results_regular_CPLEX", "results_large_BCH"])
cplex = paper_results["results_regular_CPLEX"]
bch = paper_results["results_regular_BCH"]
large = paper_results["results_large_BCH"]
large["ID_instance"] = large["ID_instance"].apply(lambda x:  50+x)

bch = bch[bch["battery_charging"] == "constant-time"]
cplex = cplex[cplex["battery_charging"] == "constant-time"]
large = large[large["battery"] == "constant-time"]
# Add a method column for each dataframe
new["method"] = new.apply(lambda x: "fragment", axis=1)
cplex["method"] = cplex.apply(lambda x: "cplex", axis=1)
bch["method"] = cplex.apply(lambda x: "bch", axis=1)
large["method"] = large.apply(lambda x: "bch", axis=1)
new.rename(columns={"objective": "objective_value"}, inplace=True)


/tmp/ipykernel_22433/99918460.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_22433/99918460.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_22433/99918460.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



20    51
21    52
22    53
23    54
24    55
25    56
26    57
27    58
28    59
29    60
30    61
31    62
32    63
33    64
34    65
35    66
36    67
37    68
38    69
39    70
Name: ID_instance, dtype: int64

In [73]:
def get_instance_id(label: str):
    try:
        data = json.load(open(f"data/instances_regular/{label}.json"))
        return data["ID"]
    except:
        print(f"No data for instance {label} found", label[-1])
        return 50 + int(label[-1])
    
new["ID_instance"] = new["label"].apply(get_instance_id)



No data for instance I-7-7-1000-03 found 3
No data for instance I-7-7-1000-05 found 5
No data for instance I-7-7-1000-01 found 1
No data for instance I-7-7-1000-02 found 2
No data for instance I-7-7-1000-04 found 4


In [74]:
new["ID_instance"]

0     17
1     25
2      8
3     11
4     18
5      9
6     29
7     10
8     30
9     27
10    16
11     4
12    26
13     1
14    23
15    22
16    14
17    12
18    13
19    20
20     7
21    19
22     3
23     2
24    21
25    28
26    15
27     6
28     5
29    24
30    53
31    55
32    51
33    52
34    54
Name: ID_instance, dtype: int64

In [104]:
new = new[["ID_instance", "method", "objective_value", "runtime"]]
cplex = cplex[["ID_instance", "method", "objective_value", "runtime"]]
bch = bch[["ID_instance", "method", "objective_value", "runtime"]]
large = large[["ID_instance", "method", "objective_value", "runtime"]]
#concatenate these dfs
result_df = pd.concat([new, cplex, bch, large])
result_df.groupby(["ID_instance"])

differences = []
{
    "ID": ...,
    "runtime_diff": ...,
    "comparator": "fragment-cplex"
}
for name, group in result_df.groupby(["ID_instance"]):
    name = name[0]
    # if name > 50:
    #      print(name)
    #      print(group)
    # else:
    #     continue
    # print(name)
    # assert bch and fragment have the same objective value
    try:
        fragment_runtime = group[group["method"] == "fragment"]["runtime"].values[0]
        bch_runtime = group[group["method"] == "bch"]["runtime"].values[0]
        assert group[group["method"] == "fragment"]["objective_value"].values[0] == group[group["method"] == "bch"]["objective_value"].values[0]

        differences += [
            {
                "ID": name,
                "runtime_diff": fragment_runtime - bch_runtime,
                "runtime_percentage": (fragment_runtime - bch_runtime )/ bch_runtime * 100,
                "comparator": "fragment-bch"
            }
        ]
        
        # differences += [
        #     {
        #         "ID": name,
        #         "runtime_diff": fragment_runtime - group[group["method"] == "cplex"]["runtime"].values[0],
        #         "runtime_percentage": (fragment_runtime - group[group["method"] == "cplex"]["runtime"].values[0]) / group[group["method"] == "cplex"]["runtime"].values[0]*100,
        #         "comparator": "fragment-cplex"
        #     },
        # ]
    except:
        print("missing data for instance", name)
   
    #compute the difference between the method column for fragment and cplex, fragment and bch
 

missing data for instance 56
missing data for instance 57
missing data for instance 58
missing data for instance 59
missing data for instance 60
missing data for instance 61
missing data for instance 62
missing data for instance 63
missing data for instance 64
missing data for instance 65
missing data for instance 66
missing data for instance 67
missing data for instance 68
missing data for instance 69
missing data for instance 70


In [105]:
result_df[51 == result_df["ID_instance"]]

,ID_instance,method,objective_value,runtime
32,51,fragment,18.0,15.809323
20,51,bch,18.0,46.630000


In [106]:
diff_df = pd.DataFrame(differences)
len(diff_df)
diff_df["ID"]
px.scatter(diff_df, x="ID", y="runtime_diff", color="comparator", title="Runtime change from paper methods").show()

In [107]:
px.scatter(diff_df, x="ID", y="runtime_percentage", color="comparator", title="Runtime % change in fragment from paper methods").show()

In [ ]:
fragment_file = r'data/instances_regular/fragments/f-I-5-5-200-10.json'
fragment_file = r'data/instances_large/fragments/f-I-7-7-1000-01.json'
fragment_data = json.load(open(fragment_file))
# instance_data = json.load(open('data/instances_regular/I-1-1-50-01.json'))
print(len(fragment_data['fragments']), len(fragment_data['contracted_fragments']),  len(fragment_data['contracted_fragments'])/len(fragment_data['fragments']) *100)

In [ ]:
from visualiser import visualise_timed_network
from constants import TimedDepot

### Arrival cases


In [ ]:
# Create 3 timed depots with same id, 5 time units apart
timed_depots = [TimedDepot(time=0, id=0), TimedDepot(id=0, time=5), TimedDepot(time=10, id=0), TimedDepot(time=15, id=0)]
fig = visualise_timed_network(timed_depots, [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])

#compressed
fig = visualise_timed_network(timed_depots[:1]+timed_depots[-1:], [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])
inbound = False
for td in timed_depots:
    # inbound
    # if inbound:
    if td.time > 9:
        x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
    else:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
    

    fig.add_annotation(
        x=x,
        y=y,
        ax=ax,
        ay=ay,
        xref='x',
        yref='y',
        axref='x',
        ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=2,  # Increase the size of the arrowhead
        arrowwidth=2,  # Increase the width of the arrow
        arrowcolor='#636363'
    )
    if td.time == 10:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
        # x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
        fig.add_annotation(
            x=x,
            y=y,
            ax=ax,
            ay=ay,
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=2,  # Increase the size of the arrowhead
            arrowwidth=2,  # Increase the width of the arrow
            arrowcolor='#636363'
        )

fig.write_image("images/time_space_compressed_two_in_one_correct.png")
fig.show()

In [20]:
from fragment_generation import ConstantFragmentGenerator
from constants import *

In [18]:
gen = ConstantFragmentGenerator("data/instances_regular/I-5-5-200-07.json")
gen.generate_fragments(file="data/instances_regular/fragments/f-I-5-5-200-07.json")
print(gen.fragments_by_id[2234])
gen.generate_timed_network()
gen.build_model()
prior_solution, solution_routes = gen.read_solution(instance_type="regular")
print(gen.timed_depots_by_fragment_id[7869])
print(gen.timed_depots_by_fragment_id[2234])

frags = gen.timed_fragments_by_timed_depot[TimedDepot(time=272,id=0)]

sol_route = gen.get_validated_timed_solution(prior_solution)


Fragment(id=2234, jobs=(Job(id=2, start_time=22, end_time=43, charge=28, building_start_id=1, building_end_id=5, start_location=(138, 193), end_location=(161, 64)),), start_time=22, end_time=50, start_depot_id=1, end_depot_id=2, charge=None)
[D1>7>S207>18>S209>26>S208>39>S206>53>S206>69>S207>83>S209>100>S206>131>141>S208>151>S207>165>S206>175>S207>193>S207>205>S206>D1]
['D1', '7', 'S207', '18', 'S209', '26', 'S208', '39', 'S206', '53', 'S206', '69', 'S207', '83', 'S209', '100', 'S206', '131', '141', 'S208', '151', 'S207', '165', 'S206', '175', 'S207', '193', 'S207', '205', 'S206', 'D1']
[D1>S209>9>S206>37>S206>51>S207>68>S209>92>S207>115>S207>127>S208>139>S207>150>S206>163>S206>192>202>S206>D1]
['D1', 'S209', '9', 'S206', '37', 'S206', '51', 'S207', '68', 'S209', '92', 'S207', '115', 'S207', '127', 'S208', '139', 'S207', '150', 'S206', '163', 'S206', '192', '202', 'S206', 'D1']
[D1>S209>10>S207>20>S208>28>S209>108>S207>118>S208>130>S208>143>S206>153>S208>164>S208>173>S207>182>S208>196>

In [28]:
sol_frags = [f for r in sol_route for f in r if isinstance(f, Fragment) and f.id in [a.id for a in frags]]
sol_frags

[Fragment(id=10110, jobs=(Job(id=47, start_time=272, end_time=306, charge=35, building_start_id=0, building_end_id=5, start_location=(86, 58), end_location=(191, 50)),), start_time=271, end_time=325, start_depot_id=0, end_depot_id=0, charge=None),
 Fragment(id=3902, jobs=(Job(id=45, start_time=258, end_time=303, charge=41, building_start_id=0, building_end_id=5, start_location=(86, 58), end_location=(89, 1)),), start_time=257, end_time=312, start_depot_id=0, end_depot_id=1, charge=None),
 Fragment(id=12496, jobs=(Job(id=48, start_time=272, end_time=295, charge=21, building_start_id=0, building_end_id=5, start_location=(105, 106), end_location=(74, 130)),), start_time=271, end_time=305, start_depot_id=0, end_depot_id=1, charge=None)]

In [29]:
for r in sol_route:
    if any(f in r for f in sol_frags):
        print(r)

[TimedDepot(time=62, id=0), Fragment(id=2378, jobs=(Job(id=1, start_time=15, end_time=64, charge=70, building_start_id=0, building_end_id=5, start_location=(105, 106), end_location=(69, 41)),), start_time=14, end_time=73, start_depot_id=0, end_depot_id=1, charge=None), TimedDepot(time=79, id=1), TimedDepot(time=86, id=1), Fragment(id=9816, jobs=(Job(id=12, start_time=85, end_time=105, charge=19, building_start_id=1, building_end_id=5, start_location=(69, 161), end_location=(182, 82)),), start_time=84, end_time=124, start_depot_id=1, end_depot_id=3, charge=None), TimedDepot(time=142, id=3), Fragment(id=5638, jobs=(Job(id=20, start_time=141, end_time=161, charge=21, building_start_id=3, building_end_id=5, start_location=(51, 39), end_location=(157, 84)),), start_time=139, end_time=168, start_depot_id=3, end_depot_id=2, charge=None), TimedDepot(time=185, id=2), TimedDepot(time=190, id=2), Fragment(id=14903, jobs=(Job(id=33, start_time=188, end_time=209, charge=28, building_start_id=2, bui

In [4]:
old_charge = pd.read_csv('new_results.csv', index_col=0)
fixed = pd.read_csv('fixed_charge_cost.csv', index_col=0)
fixed["method"] = fixed.apply(lambda x: "fixed_charge", axis=1)

In [5]:
df = pd.concat([old_charge, fixed])

In [9]:
groupedby = df.groupby(["label"])
for name, group in groupedby:
    print(name)
    # print(group)\
    # assert objective is the same

    # show the duifference in num fragments
    print("num fragments", group["num_fragments"].values)

    print("")

('I-1-1-50-01',)
num fragments [668 484]

('I-1-1-50-02',)
num fragments [2604 1166]

('I-1-1-50-03',)
num fragments [1952 1030]

('I-1-1-50-04',)
num fragments [668 668 561]

('I-1-1-50-05',)
num fragments [552 516]

('I-1-1-50-06',)
num fragments [708 543]

('I-1-1-50-07',)
num fragments [998 702]

('I-1-1-50-08',)
num fragments [1236 1236  719]

('I-1-1-50-09',)
num fragments [614 614 476]

('I-1-1-50-10',)
num fragments [1528 1528  911]

('I-3-3-100-01',)
num fragments [9510 9510 3623]

('I-3-3-100-02',)
num fragments [42072  4859]

('I-3-3-100-03',)
num fragments [7569 3456]

('I-3-3-100-04',)
num fragments [14961  4609]

('I-3-3-100-05',)
num fragments [9105 3067]

('I-3-3-100-06',)
num fragments [39315 39315  5210]

('I-3-3-100-07',)
num fragments [8040 8040 8040 8040 2783]

('I-3-3-100-08',)
num fragments [128328 128328   6125]

('I-3-3-100-09',)
num fragments [21384  4554]

('I-3-3-100-10',)
num fragments [16116  3600]

('I-5-5-200-01',)
num fragments [642179  24063]

('I-5-5-

In [2]:
df = pd.read_xml(open('data/instances_large.xml', 'r'))

In [8]:
df
# check each column e_j, s_j, c_j tripStartLocations has the same length
for _, row in df.iterrows():
    print(len(row["tripStartLocations"]), len(row["tripEndLocations"]), len(row["e_j"]), len(row["s_j"]), len(row["c_j"]))
    print(type(row["e_j"]))

17310 17906 4802 4792 3001
<class 'str'>
16725 17983 4792 4781 3001
<class 'str'>
16821 17850 4790 4772 3001
<class 'str'>
16561 17945 4794 4782 3001
<class 'str'>
16896 17915 4799 4787 3001
<class 'str'>
33674 35853 9858 9837 6001
<class 'str'>
33510 35746 9801 9786 6001
<class 'str'>
33675 35801 9815 9798 6001
<class 'str'>
33629 35488 9820 9798 6001
<class 'str'>
32839 36077 9793 9778 6001
<class 'str'>
48316 53374 15773 15750 9001
<class 'str'>
49649 53471 15817 15792 9001
<class 'str'>
48356 53283 15767 15749 9001
<class 'str'>
49718 53982 15780 15757 9001
<class 'str'>
50740 53996 15798 15781 9001
<class 'str'>
67623 71449 21767 21744 12001
<class 'str'>
66207 72043 21798 21779 12001
<class 'str'>
66841 71435 21810 21790 12001
<class 'str'>
66327 71837 21797 21781 12001
<class 'str'>
68746 71600 21870 21847 12001
<class 'str'>
